In [1]:
!pip install albumentations==1.4.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: albumentations
    Found existing installation: albumentations 2.0.8
    Uninstalling albumentations-2.0.8:
      Successfully uninstalled albumentations-2.0.8


In [2]:
!cp -r /kaggle/input/test-rnsa-2024-code/Kaggle-2024-RSNA-Pub-main /kaggle/working/code

In [3]:
import os
import json
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
kaggle_json = user_secrets.get_secret("kaggle")

# Make config folder
os.makedirs("/root/.config/kaggle", exist_ok=True)

with open("/root/.config/kaggle/kaggle.json", "w") as f:
    f.write(kaggle_json)

os.chmod("/root/.config/kaggle/kaggle.json", 0o600)

In [4]:
%cd /kaggle/working/code
%env PYTHONPATH=/kaggle/working/code:$PYTHONPATH

# !bash scripts/run.sh

/kaggle/working/code
env: PYTHONPATH=/kaggle/working/code:$PYTHONPATH


In [5]:
import os
import zipfile
import pathlib
from kaggle.api.kaggle_api_extended import KaggleApi

from src.utils import load_settings


def main():
    settings = load_settings()
    raw_data_dir = settings.raw_data_dir
    pretrained_checkpoint_dir = settings.pretrained_checkpoint_dir
    raw_data_dir.mkdir(parents=True, exist_ok=True)

    api = KaggleApi()
    api.authenticate()

    datasets = [
        {
            'type': 'dataset',
            'name': 'brendanartley/lumbar-coordinate-pretraining-dataset',
            'extract_dir': raw_data_dir / 'lumbar_coordinate_dataset'
        },
        {
            'type': 'competition_symlink',
            'name': 'rsna-2024-lumbar-spine-degenerative-classification',
            'extract_dir': raw_data_dir / 'rsna-2024-lumbar-spine-degenerative-classification'
        },
        ## NO MORE STORAGE :))))
        # {
        #     'type': 'dataset',
        #     'name': 'akinosora/rsna2024-lsdc-3rd-models-pub',
        #     'extract_dir': pretrained_checkpoint_dir,
        # },
    ]

    for item in datasets:
        if item['type'] == 'dataset':
            print(f"Downloading dataset {item['name']}")
            api.dataset_download_files(item['name'], path=str(raw_data_dir), quiet=False)
            zip_filename = item['name'].split('/')[-1] + '.zip'

            zip_file = raw_data_dir / zip_filename
            extract_dir = item['extract_dir']
            print(f'Extracting {zip_file} to {extract_dir}')
            with zipfile.ZipFile(zip_file, 'r') as zip_ref:
                zip_ref.extractall(extract_dir)
            print(f'Removing {zip_file}')
            os.remove(zip_file)

        elif item['type'] == 'competition_symlink':
            src_path = pathlib.Path(f"/kaggle/input/{item['name']}")
            dst_path = item['extract_dir']

            if dst_path.exists():
                print(f"Already exists: {dst_path}")
            else:
                print(f"Linking competition dataset {src_path} -> {dst_path}")
                os.symlink(src_path, dst_path)


if __name__ == '__main__':
    main()


Dataset URL: https://www.kaggle.com/datasets/brendanartley/lumbar-coordinate-pretraining-dataset


100%|██████████| 717M/717M [00:00<00:00, 1.23GB/s]



Extracting input/lumbar-coordinate-pretraining-dataset.zip to input/lumbar_coordinate_dataset
Removing input/lumbar-coordinate-pretraining-dataset.zip
Linking competition dataset /kaggle/input/rsna-2024-lumbar-spine-degenerative-classification -> input/rsna-2024-lumbar-spine-degenerative-classification


In [6]:
# # !mkdir -p input/rsna-2024-lumbar-spine-degenerative-classification
# # !cp /kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train.csv input/rsna-2024-lumbar-spine-degenerative-classification/train.csv
# !mkdir -p input/rsna-2024-lumbar-spine-degenerative-classification
# !cp /kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/*.csv input/rsna-2024-lumbar-spine-degenerative-classification/


In [7]:
!ln -s /kaggle/input/rsna2024-lsdc-3rd-models-pub /kaggle/working/code/input/pretrained_checkpoint_dir

In [8]:
!python src/utils/generate_kfold.py


9719 2249
9737 2092
9735 2258
9733 2169
9733 2263
add {'study_id': 3008676218, 'fold': 0}


In [9]:
# !python3 src/stage1/moyashii/tools/create_dataset_v2.py moyashii 2024


In [10]:
### Moyashii
# Create sagittal dataset v2 and train the sagittal keypoint models
!python3 src/stage1/moyashii/tools/create_dataset_v2.py moyashii 2024
!python3 src/stage1/moyashii/sagittal/train.py src/stage1/moyashii/sagittal/config/v2_0008_config.yaml 0008 --dst_root stage1/moyashii/v2/0008

100%|██████████████████████████████████████| 1121/1121 [00:02<00:00, 451.45it/s]
config: src/stage1/moyashii/sagittal/config/v2_0008_config.yaml
experiment: 0008
dst_root: stage1/moyashii/v2/0008
options: []
dst_root: output/stage1/moyashii/v2/0008
dataloader:
  batch_size: 32
  num_workers: 8
dataset:
  heatmap_size:
  - 152
  - 152
  image_root: input/processed/moyashii/keypoint_datasets/v2
  label_csv_path: input/processed/moyashii/keypoint_datasets/v2/train_all_5folds.csv
  name: RSNA2024KeypointDatasetTrainV1
  stride: 2
device: 0
epochs: 10
folds:
- 0
- 1
- 2
- 3
- 4
metrics:
  percentile: 99.8
  score_name: acc(per)@0.02
  stride: 2
model:
  loss:
    name: BFWithLogitsLoss
  timm_model:
    drop_path_rate: 0.2
    drop_rate: 0.3
    features_only: true
    in_chans: 1
    model_name: tf_efficientnet_b6.ns_jft_in1k
    out_indices:
    - 0
    - 1
    - 2
    - 3
    - 4
    pretrained: true
num_folds: 5
optimizer:
  betas:
  - 0.9
  - 0.999
  lr: 0.0002
  name: AdamW
  weight_d

In [11]:
# Create axial dataset v5 and train the axial keypoint models
!python3 src/stage1/moyashii/tools/create_dataset_v5.py moyashii 2024
!python3 src/stage1/moyashii/axial/train.py src/stage1/moyashii/axial/config/v5_0003_config.yaml 0003 --dst_root stage1/moyashii/v5/0003

100%|███████████████████████████████████████| 1792/1792 [15:34<00:00,  1.92it/s]
config: src/stage1/moyashii/axial/config/v5_0003_config.yaml
experiment: 0003
dst_root: stage1/moyashii/v5/0003
options: []
dst_root: output/stage1/moyashii/v5/0003
dataloader:
  batch_size: 32
  num_workers: 8
dataset:
  heatmap_size:
  - 152
  - 152
  image_root: input/processed/moyashii/keypoint_datasets/v5
  label_csv_path: input/processed/moyashii/keypoint_datasets/v5/train_10folds.csv
  stride: 4
device: 0
epochs: 10
folds:
- 0
- 1
- 2
- 3
- 4
metrics:
  percentile: 99.8
  score_name: acc(per)@0.02
  stride: 4
model:
  loss:
    name: BFWithLogitsLoss
  num_classes: 1
  timm_model:
    drop_path_rate: 0.2
    drop_rate: 0.3
    features_only: true
    in_chans: 1
    model_name: tf_efficientnet_b4.in1k
    out_indices:
    - 1
    - 2
    - 3
    - 4
    pretrained: true
num_folds: 5
optimizer:
  betas:
  - 0.9
  - 0.999
  lr: 0.0002
  name: AdamW
  weight_decay: 0.01
scheduler:
  max_lr: 0.0002
  na